In [1]:
import pandas as pd
import json
import numpy
import geopy.distance
import math
import sys


In [23]:
dataset = pd.read_csv("partial_subtracks_with_extra_points_after_dividing_only_city_centre.csv")

In [24]:
# calculate the distance between the two points


def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    if dist > 30:
        return -1
    else:
        return 1

In [25]:
def actual_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    return dist

In [26]:
gap_penalty = -1

def create_matrix(track1, track2):
    matrix = numpy.zeros((len(track1) + 1, len(track2) + 1))
    for i in range(0, len(track1) + 1):
        matrix[i, 0] = gap_penalty * i
    for j in range(0, len(track2) + 1):
        matrix[0, j] = gap_penalty * j

    for i in range(1, len(track1) + 1):
        t1 = track1[i - 1]
        for j in range(1, len(track2) + 1):
            t2 = track2[j - 1]
            match = matrix[i - 1, j - 1] + point_distance(t1, t2)
            delete = matrix[i - 1, j] + gap_penalty
            insert = matrix[i, j - 1] + gap_penalty
            matrix[i, j] = max(match, max(delete, insert))

    return matrix


track1 = dataset.loc[dataset["id"] == "T2"].values.tolist()
track2 = dataset.loc[dataset["id"] == "T18"].values.tolist()
matrix = create_matrix(track1, track2)
print(matrix)

[[   0.   -1.   -2. ... -207. -208. -209.]
 [  -1.   -1.   -2. ... -207. -208. -209.]
 [  -2.   -2.   -2. ... -207. -208. -209.]
 ...
 [ -77.  -77.  -77. ... -175. -176. -177.]
 [ -78.  -78.  -78. ... -175. -176. -177.]
 [ -79.  -79.  -79. ... -175. -176. -177.]]


In [27]:
def backtrack(matrix, track1, track2):

    i = len(track1)
    j = len(track2)

    a1 = []

    while i > 0 and j > 0:
        if (
            matrix[i, j] == matrix[i - 1, j - 1] + 1
            and point_distance(
                (track1[i - 1][0], track1[i - 1][1]),
                (track2[j - 1][0], track2[j - 1][1]),
            )
            > 0
        ):
            a1.insert(0, [track1[i - 1][0], track1[i - 1][1], track2[0][2]])
            i = i - 1
            j = j - 1
        elif matrix[i, j] == matrix[i - 1, j - 1] - 1:
            i = i - 1
            j = j - 1
        elif matrix[i, j] == matrix[i - 1, j] - 1:
            i = i - 1
        elif matrix[i, j] == matrix[i, j - 1] - 1:
            j = j - 1
        else:
            print("5")
            a1 = []
            break
    return a1


align = backtrack(matrix, track1, track2)
print(align)

[[41.14605, -8.610596999999999, 'T18'], [41.146191, -8.610443999999998, 'T18'], [41.146191, -8.610444, 'T18'], [41.1462855, -8.610277499999999, 'T18'], [41.14638, -8.610111, 'T18'], [41.1464745, -8.609944500000001, 'T18'], [41.14656900000001, -8.609778000000002, 'T18'], [41.14666350000001, -8.609611500000003, 'T18'], [41.14675800000001, -8.609445000000004, 'T18'], [41.146758, -8.609445000000001, 'T18'], [41.146878, -8.609262000000001, 'T18'], [41.146998, -8.609079000000001, 'T18'], [41.147118000000006, -8.608896000000001, 'T18'], [41.147118, -8.608896, 'T18'], [41.147127, -8.608968, 'T18'], [41.1473295, -8.6088375, 'T18']]


In [29]:
def check_length(matrix):
    longest_alignment = numpy.max(matrix)
    if longest_alignment <= 3:
        return False
    else:
        return True

In [30]:
track_ids = pd.read_csv("partial_subtracks_ids_add_points_after_dividing.csv").values.tolist()
#track_ids = pd.read_csv("partial_subtracks_ids.csv").values.tolist()

In [38]:

def Needleman_Wunsch(dataset, trackid, nr):
    track = dataset.loc[dataset["id"] == trackid]
    track = pd.DataFrame.drop_duplicates(track)
    print("track1: ", trackid)
    alignments = []
    for j in range(nr + 1, len(track_ids)):
        #print("track2: ", track_ids[j][0])
        next_track = dataset.loc[dataset["id"] == track_ids[j][0]]
        matrix = create_matrix(track.values.tolist(), next_track.values.tolist())
        alignment = backtrack(matrix, track.values.tolist(), next_track.values.tolist())
        if (len(alignment) > 3):
            alignments.extend(alignment)
    return alignments


align = Needleman_Wunsch(dataset, "T2", 1)

track1:  T2


In [39]:
print(align)

[[41.14605, -8.610596999999999, 'T18'], [41.146191, -8.610443999999998, 'T18'], [41.146191, -8.610444, 'T18'], [41.1462855, -8.610277499999999, 'T18'], [41.14638, -8.610111, 'T18'], [41.1464745, -8.609944500000001, 'T18'], [41.14656900000001, -8.609778000000002, 'T18'], [41.14666350000001, -8.609611500000003, 'T18'], [41.14675800000001, -8.609445000000004, 'T18'], [41.146758, -8.609445000000001, 'T18'], [41.146878, -8.609262000000001, 'T18'], [41.146998, -8.609079000000001, 'T18'], [41.147118000000006, -8.608896000000001, 'T18'], [41.147118, -8.608896, 'T18'], [41.147127, -8.608968, 'T18'], [41.1473295, -8.6088375, 'T18'], [41.14557, -8.610876000000001, 'T32'], [41.145579000000005, -8.610858, 'T32'], [41.145768, -8.610903, 'T32'], [41.145909, -8.61075, 'T32'], [41.14605, -8.610596999999999, 'T32'], [41.146191, -8.610443999999998, 'T32'], [41.146191, -8.610444, 'T32'], [41.1462855, -8.610277499999999, 'T32'], [41.14638, -8.610111, 'T32'], [41.1464745, -8.609944500000001, 'T32'], [41.146

In [32]:
def create_list_of_points(tracknr, alignments):
    
    track = dataset.loc[dataset["id"] == tracknr]
    track = pd.DataFrame.drop_duplicates(track)
    points = pd.DataFrame(columns=["lat", "lon", "ids"])
    align = pd.DataFrame(alignments, columns=["lat", "lon", "id"])

    for i in range(len(track)):
        lat = align.loc[align["lat"] == track.iloc[i]["lat"]]
        points.loc[i] = [
            track.iloc[i]["lat"],
            track.iloc[i]["lon"],
            lat["id"].values.tolist(),
        ]
    return points

#points = create_list_of_points("T1", align)
#print(points)

In [33]:
def remove_infrequent_points(points):
    point_list = pd.DataFrame(points, columns=["lat", "lon", "ids"])
    for i in range(len(point_list)):
        val = len(point_list.loc[i]["ids"])
        point_list.at[i, "ids"] = val
    return point_list
#new_points = remove_infrequent_points(points)
#print(new_points)

In [34]:
def gather_sequences(points):
    i = 0
    final_records = pd.DataFrame(columns=["polyline", "value"])
    for group in range(points.iloc[0]["group"], -1, -1):
        records = pd.DataFrame(columns=["polyline", "value"])
        sequence = []
        value = 0
        while i < len(points) and points.iloc[i]["group"] == group:
            if value == points.iloc[i]["ids"] or value == 0:
                sequence.append(
                    [
                        points.index[i],
                        points.iloc[i]["lat"],
                        points.iloc[i]["lon"],
                    ]
                )
                value = points.iloc[i]["ids"]
                i += 1
            else:
                sequence.sort()
                new_sequence = sequence.copy()
                records.loc[len(records)] = [
                    new_sequence,
                    (len(new_sequence) * 1.0) + value,
                ]
                value = points.iloc[i]["ids"]

        sequence.sort()
        new_sequence = sequence.copy()
        records.loc[len(records)] = [
            new_sequence,
            (len(new_sequence) * 1.0) + value,
        ]
        if len(records) < 2:
            final_records = pd.concat([final_records, records])
        else:
            highest_value = records["value"].idxmax()
            final_records = final_records.append(records.loc[highest_value])

    return final_records

#new_points['group'] = (new_points.index.to_series().diff() > 1).cumsum()
#print(new_points)
#new_points.sort_values(by=['group','ids'], ascending=False, inplace=True)
#list = gather_sequences(new_points)
#print(list)

In [107]:
numpy.set_printoptions(threshold=sys.maxsize)
print(list["polyline"].iloc[0])

[[0, 41.148522, -8.585676], [1, 41.148639, -8.585712000000001], [2, 41.148855000000005, -8.585685000000002], [3, 41.14892699999999, -8.585730000000002], [4, 41.148963, -8.585982], [5, 41.1489585, -8.586189000000001], [6, 41.148954, -8.586396], [7, 41.148837, -8.586234000000001], [8, 41.14872, -8.586072000000001], [9, 41.14872, -8.586072], [10, 41.1485454, -8.5861224], [11, 41.1483708, -8.586172800000002], [12, 41.1481962, -8.586223200000003], [13, 41.14802159999999, -8.586273600000004], [14, 41.14784699999999, -8.586324000000005], [15, 41.147847, -8.586324000000001], [16, 41.14775025, -8.586492750000001], [17, 41.1476535, -8.586661500000002], [18, 41.14755675000001, -8.586830250000002], [19, 41.14746000000001, -8.586999000000002], [20, 41.14746, -8.586999], [21, 41.147307, -8.5867875], [22, 41.147154, -8.586575999999999], [23, 41.14715400000001, -8.586575999999999], [24, 41.14709500000001, -8.586388], [25, 41.14703600000001, -8.5862], [26, 41.14697700000001, -8.586012], [27, 41.1469180

In [36]:
def hele_sullamitten(dataset):
    most_tracked_roads = pd.DataFrame(columns = ['polyline', 'value'])
    for i in range(len(track_ids)):
        track = track_ids[i][0]
        alignment = Needleman_Wunsch(dataset, track, i)
        points = create_list_of_points(track, alignment)
        new_points = remove_infrequent_points(points)
        new_points = new_points[new_points.ids > 8]
        if not new_points.empty:
            new_points['group'] = (new_points.index.to_series().diff() > 1).cumsum()
            new_points.sort_values(by=['group','ids'], ascending=False, inplace=True)
            records = gather_sequences(new_points)
            most_tracked_roads = pd.concat([most_tracked_roads, records])
            print(most_tracked_roads)
    return most_tracked_roads

res = hele_sullamitten(dataset)

track1:  T1
                                            polyline  value
1  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
track1:  T2
                                            polyline  value
1  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
track1:  T2334
track1:  T3
                                            polyline  value
1  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
track1:  T3463
track1:  T3569
                                            polyline  value
1  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1  [[1, 41.148828, 

track1:  T1684519
track1:  T1626525
track1:  T17
track1:  T1728548
                                            polyline  value
1  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0  [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
0  [[22, 41.156487, -8.582985000000004], [23, 41....   13.0
0  [[4, 41.153832, -8.582021999999998], [5, 41.15...   12.0
0  [[2, 41.148945, -8.585919], [3, 41.14893599999...   20.0
0  [[3, 41.152923, -8.581185000000001], [4, 41.15...   13.0
0  [[1, 41.144724, -8.606673], [4, 41.144724, -8....   21.0
0  [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
0  [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0  [[36, 41.15640600000001, -8.582985000000003], ...   11.0
0  [[28, 41.165118, -8.588978999999998], [29, 41....   13.0
0  [[5, 41.14897200000001, -8.586

                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   13.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   12.0
0   [[2, 41.148945, -8.585919], [3, 41.14893599999...   20.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   13.0
0   [[1, 41.144724, -8.606673], [4, 41.144724, -8....   21.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   11.0
0   [[28, 41.165118, -8.588978999999998], [29, 41....   13.0
0   [[5, 41.14897200000001, -8.586063000000001], [...   32.0
0   [[8, 41.150844, -8.5

                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   13.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   12.0
0   [[2, 41.148945, -8.585919], [3, 41.14893599999...   20.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   13.0
0   [[1, 41.144724, -8.606673], [4, 41.144724, -8....   21.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   11.0
0   [[28, 41.165118, -8.588978999999998], [29, 41....   13.0
0   [[5, 41.14897200000001, -8.586063000000001], [...   32.0
0   [[8, 41.150844, -8.5

track1:  T305922
track1:  T31
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   13.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   12.0
0   [[2, 41.148945, -8.585919], [3, 41.14893599999...   20.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   13.0
0   [[1, 41.144724, -8.606673], [4, 41.144724, -8....   21.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   11.0
0   [[28, 41.165118, -8.588978999999998], [29, 41....   13.0
0   [[5, 41.14897200000001, -8.586063000000001], [...  

...

track1:  T1811084525
track1:  T181414554
track1:  T184
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
..                                                ...    ...
1   [[34, 41.147622000000005, -8.609490000000001],...   19.0
2   [[27, 41.147370000000016, -8.608554000000005],...   15.0
0   [[13, 41.146047, -8.607068999999997], [14, 41....   12.0
0   [[6, 41.144904, -8.607051000000002], [7, 41.14...   13.0
0   [[1, 41.144589, -8.606439], [2, 41.144589, -8....   11.0

[240 rows x 2 columns]
track1:  T184204628
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 

track1:  T211885529
track1:  T211575539
track1:  T211565544
track1:  T211155584
track1:  T21375604
track1:  T214
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
..                                                ...    ...
0   [[111, 41.152428, -8.634834000000005], [112, 4...   47.0
0                       [[107, 41.152563, -8.633997]]   21.0
0                       [[103, 41.152644, -8.633268]]   20.0
15  [[4, 41.147883, -8.619966], [5, 41.147865, -8....   99.0
0   [[0, 41.147991, -8.619831], [1, 41.147991, -8....   27.0

[278 rows x 2 columns]
track1:  T215
track1:  T2171135676
track1:  T217235685
                                             polyline  value
1   [[3, 41.148

track1:  T237826821
track1:  T237676831
track1:  T23716835
track1:  T23736840
track1:  T238
track1:  T238696851
track1:  T239
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
..                                                ...    ...
0                         [[4, 41.148234, -8.624088]]   12.0
0                         [[2, 41.148216, -8.624079]]   31.0
0   [[137, 41.167845, -8.606474999999989], [138, 4...   14.0
0   [[26, 41.149539, -8.610327], [27, 41.149723500...   15.0
0   [[17, 41.148107, -8.610863000000002], [18, 41....   17.0

[318 rows x 2 columns]
track1:  T239516891
track1:  T240
                                             polyline  value
1   [[3, 41.14892699999

track1:  T255
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
..                                                ...    ...
0   [[105, 41.152509, -8.628489000000005], [106, 4...   15.0
3   [[37, 41.1476682857143, -8.620142142857146], [...   73.0
0   [[50, 41.153823, -8.609885999999996], [51, 41....   12.0
0   [[51, 41.145939, -8.612397000000001], [52, 41....   14.0
0                        [[44, 41.145939, -8.613612]]   13.0

[344 rows x 2 columns]
track1:  T255107308
track1:  T256
track1:  T25617330
track1:  T256697335
track1:  T257
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.14989900

track1:  T280577807
track1:  T281
track1:  T28117819
track1:  T28117823
track1:  T282
track1:  T28217836
                                             polyline  value
1   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   17.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   29.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   13.0
..                                                ...    ...
0   [[36, 41.150772, -8.610111], [37, 41.150772, -...   18.0
0   [[72, 41.148603, -8.625068999999998], [73, 41....   12.0
0   [[24, 41.147865, -8.610642], [25, 41.147865, -...   11.0
0                        [[12, 41.145948, -8.612397]]   11.0
0                         [[10, 41.145948, -8.61246]]   11.0

[366 rows x 2 columns]
track1:  T283
track1:  T28317852
track1:  T284
                                             polyline  value
1   [[3, 41.14892699999999, -8.

In [37]:
pd.DataFrame.to_csv(res, "./Needleman_Wunsch_result_final_final.csv", index=True)

In [21]:
print(res.loc[0].polyline)

[[8, 41.14872, -8.586072000000001], [9, 41.14872, -8.586072]]


In [ ]:
def align_tracks(track1, track2, gap_penalty):
    """ Needleman-Wunsch algorithm adapted for gps tracks. """
    
    _log.info("Aligning tracks")

    def similarity(p1, p2):
        d = gpxpy.geo.distance(p1.latitude, p1.longitude, p1.elevation,
                               p2.latitude, p2.longitude, p2.elevation)
        return -d

    # construct f-matrix
    f = numpy.zeros((len(track1), len(track2)))
    for i in range(0, len(track1)):
        f[i][0] = gap_penalty * i
    for j in range(0, len(track2)):
        f[0][j] = gap_penalty * j
    for i in range(1, len(track1)):
        t1 = track1[i]
        for j in range(1, len(track2)):
            t2 = track2[j]
            match = f[i-1][j-1] + similarity(t1, t2)
            delete = f[i-1][j] + gap_penalty<
            insert = f[i][j-1] + gap_penalty
            f[i, j] = max(match, max(delete, insert))

    # backtrack to create alignment
    a1 = []
    a2 = []
    i = len(track1) - 1
    j = len(track2) - 1
    while i > 0 or j > 0:
        if i > 0 and j > 0 and \
           f[i, j] == f[i-1][j-1] + similarity(track1[i], track2[j]):
            a1.insert(0, track1[i])
            a2.insert(0, track2[j])
            i -= 1
            j -= 1
        elif i > 0 and f[i][j] == f[i-1][j] + gap_penalty:
            a1.insert(0, track1[i])
            a2.insert(0, None)
            i -= 1
        elif j > 0 and f[i][j] == f[i][j-1] + gap_penalty:
            a1.insert(0, None)
            a2.insert(0, track2[j])
            j -= 1
    return a1, a2

align_tracks()